In [52]:
import os
import numpy as np
import pandas as pd
import pylab as pl
import datetime as dt

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GMM
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
%matplotlib inline

pd.options.display.max_columns = None
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [85]:
stations = pd.read_csv('Clean_Data/citibike_station.csv')

In [86]:
stations.head()

,capacity,lat,lon,name,short_name,station_id
0,37,40.722174,-73.983688,E 2 St & Avenue B,5515.02,301
1,52,40.727408,-73.981420,E 10 St & Avenue A,5659.05,445
2,34,40.733143,-73.975739,E 20 St & FDR Drive,5886.02,487
3,44,40.732219,-73.981656,1 Ave & E 16 St,5779.08,504
4,39,40.767272,-73.993929,W 52 St & 11 Ave,6926.01,72


In [87]:
realtime = pd.read_csv("Clean_Data/station_status.csv")

In [88]:
realtime.head(1)

,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,station_id
0,2018-10-29 20:42:37,32,6,18,1,281


In [89]:
realtime.shape

(1842035, 6)

In [90]:
realtime = realtime[realtime['station_id'] != 3714]
realtime = realtime[realtime['station_id'] != 3715]
realtime = realtime[realtime['station_id'] != 3716]
realtime = realtime[realtime['station_id'] != 3717]

In [91]:
realtime.shape

(1840646, 6)

In [92]:
realtime = realtime[realtime['num_bikes_available']==0]

In [93]:
realtime.shape

(243784, 6)

In [94]:
realtime['last_reported'] = realtime['last_reported'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [95]:
realtime.shape

(243784, 6)

In [96]:
realtime = realtime[realtime.apply(lambda x: x['last_reported'].year != 1970, axis=1)]

In [97]:
realtime.shape

(212419, 6)

### Empty dock rate

As each station has be record same times, we simple use the count of 0 in each hour for each station as its empty dock rate.

In [98]:
for i in range(24):
    realtimeHour = realtime[realtime.apply(lambda x: x['last_reported'].hour == i, axis=1)]
    #print(realtimeHour.head())
    empytCount = realtimeHour.groupby('station_id').count()
    empytCount = empytCount.reset_index()[[0,1]]
    
    stations = stations.merge(empytCount, how='left', left_on='station_id', right_on='station_id')
    empytCount_str = 'empytCount_' + str(i)
    stations = stations.rename(index=str, columns={"last_reported": empytCount_str})
    

In [99]:
stations.head()

,capacity,lat,lon,name,short_name,station_id,empytCount_0,empytCount_1,empytCount_2,empytCount_3,empytCount_4,empytCount_5,empytCount_6,empytCount_7,empytCount_8,empytCount_9,empytCount_10,empytCount_11,empytCount_12,empytCount_13,empytCount_14,empytCount_15,empytCount_16,empytCount_17,empytCount_18,empytCount_19,empytCount_20,empytCount_21,empytCount_22,empytCount_23
0,37,40.722174,-73.983688,E 2 St & Avenue B,5515.02,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,39.0,48.0,33.0,35.0,39.0,26.0,18.0,15.0,2.0,NaN,5.0
1,52,40.727408,-73.981420,E 10 St & Avenue A,5659.05,445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,14.0,19.0,17.0,8.0,22.0,15.0,5.0,9.0,2.0,NaN,NaN
2,34,40.733143,-73.975739,E 20 St & FDR Drive,5886.02,487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,28.0,35.0,39.0,27.0,31.0,12.0,12.0,3.0,1.0,NaN,1.0
3,44,40.732219,-73.981656,1 Ave & E 16 St,5779.08,504,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,37.0,44.0,34.0,30.0,24.0,21.0,13.0,13.0,4.0,NaN,2.0
4,39,40.767272,-73.993929,W 52 St & 11 Ave,6926.01,72,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,11.0,12.0,5.0,3.0,10.0,13.0,14.0,6.0,15.0,19.0,15.0,5.0


In [100]:
stations = stations.fillna(0)

In [101]:
stations.head(5)

,capacity,lat,lon,name,short_name,station_id,empytCount_0,empytCount_1,empytCount_2,empytCount_3,empytCount_4,empytCount_5,empytCount_6,empytCount_7,empytCount_8,empytCount_9,empytCount_10,empytCount_11,empytCount_12,empytCount_13,empytCount_14,empytCount_15,empytCount_16,empytCount_17,empytCount_18,empytCount_19,empytCount_20,empytCount_21,empytCount_22,empytCount_23
0,37,40.722174,-73.983688,E 2 St & Avenue B,5515.02,301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,39.0,48.0,33.0,35.0,39.0,26.0,18.0,15.0,2.0,0.0,5.0
1,52,40.727408,-73.981420,E 10 St & Avenue A,5659.05,445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,14.0,19.0,17.0,8.0,22.0,15.0,5.0,9.0,2.0,0.0,0.0
2,34,40.733143,-73.975739,E 20 St & FDR Drive,5886.02,487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,28.0,35.0,39.0,27.0,31.0,12.0,12.0,3.0,1.0,0.0,1.0
3,44,40.732219,-73.981656,1 Ave & E 16 St,5779.08,504,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,37.0,44.0,34.0,30.0,24.0,21.0,13.0,13.0,4.0,0.0,2.0
4,39,40.767272,-73.993929,W 52 St & 11 Ave,6926.01,72,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,11.0,12.0,5.0,3.0,10.0,13.0,14.0,6.0,15.0,19.0,15.0,5.0


In [102]:
stations.to_csv('Clean_Data/emptyCount.csv', index=False)

In [76]:
#stations.drop(['start station id', 'end station id'], axis=1, inplace=True)
#empytCount_str = 'empytCount_' + str(i)
#stations = stations.rename(index=str, columns={"last_reported": empytCount_str})

In [77]:
stations.head()

,capacity,lat,lon,name,short_name,station_id,empytCount_0
0,37,40.722174,-73.983688,E 2 St & Avenue B,5515.02,301,NaN
1,52,40.727408,-73.981420,E 10 St & Avenue A,5659.05,445,NaN
2,34,40.733143,-73.975739,E 20 St & FDR Drive,5886.02,487,NaN
3,44,40.732219,-73.981656,1 Ave & E 16 St,5779.08,504,2.0
4,39,40.767272,-73.993929,W 52 St & 11 Ave,6926.01,72,5.0


In [22]:
#realtime[realtime['station_id']==143]

In [44]:
#realtime.groupby('station_id').count().tail(5)

In [11]:
realtime[realtime['station_id']==3717]

,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,station_id
1818344,1970-01-01 00:00:01,0,0,0,0,3717


In [21]:
stations[stations['station_id']==3712]

,capacity,lat,lon,name,short_name,station_id
817,41,40.754692,-73.997402,W 35 St & Dyer Ave,6569.08,3712
